In [16]:
import tensorflow as tf
import os
import cv2
import numpy as np
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Concatenate
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer
import pathlib
from keras.applications.vgg16 import VGG16

In [2]:
MMI_emotions =  pathlib.Path("C:/Users/cdr03/Documents/Thesis/dataset/MMI/Emotions")

In [38]:
def list_files_recursive(path, format = ".txt"):
    txt_files = set()  # create an empty set to store unique txt file names
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(format):
                file_path = os.path.join(root, file)
                txt_files.add(file_path)  # add the file path to the set
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            list_files_recursive(dir_path)  # recursively call the function on each subdirectory
    return list(txt_files)  # return a list of unique txt file names

MMI_emotion_videos = list_files_recursive(MMI_emotions, ".avi")

In [90]:
def image_to_array(img, size):
    img_array = np.array(img , dtype="float32")/255
    img_array = img_array.reshape(size, size, 1)
    # Return the numpy array
    return img_array

In [103]:
size = 48
def detect_and_crop_face(img,resize=None):
    # Load the input image
    	
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Load the Haar Cascade classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)

    # Crop the image to contain the detected face (if any)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        face_image = image[y:y+h, x:x+w]
        face_image = cv2.resize(face_image, (size, size))
        face_image = image_to_array(face_image, size)
    else:
        rotated=cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        rotated_faces = face_cascade.detectMultiScale(rotated, scaleFactor=1.1, minNeighbors=5)
        if len(rotated_faces) > 0:
            (x, y, w, h) = rotated_faces[0]
            face_image = rotated[y:y+h, x:x+w]
            face_image = cv2.resize(face_image, (size, size))
            face_image = image_to_array(face_image, size)
        else:
            face_image = None
            print("face wasn't detected")
    return face_image

In [98]:
# https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-TensorFlow-2/blob/master/Chapter08/ch8_nb1_action_recognition.ipynb

video_paths = MMI_emotion_videos
print(type(MMI_emotion_videos))
SEQUENCE_LENGTH = 10
IMG_size = (48, 48)
def frame_generator():
    np.random.shuffle(video_paths)
    for video_path in video_paths:
        frames = []
        cap = cv2.VideoCapture(video_path)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_every_frame = max(1, num_frames // SEQUENCE_LENGTH)
        current_frame = 0

        label = os.path.basename(video_path)

        max_images = SEQUENCE_LENGTH
        while True:
            success, frame = cap.read()
            if not success:
                break

            if current_frame % sample_every_frame == 0:
                # OPENCV reads in BGR, tensorflow expects RGB so we invert the order
                face_image_frame = detect_and_crop_face(frame, size)
                if face_image_frame is not None:
                    yield face_image_frame, video_path 
                else:
                    pass
            if max_images == 0:
                break
            current_frame += 1

# `from_generator` might throw a warning, expected to disappear in upcoming versions:
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#for_example_2
dataset = tf.data.Dataset.from_generator(frame_generator,
             output_types=(tf.float32, tf.string),
             output_shapes=((48, 48, 1), ()))

dataset = dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)



<class 'list'>


In [101]:
def change_input_layer(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (None, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    biases  = layer.get_weights()[1]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel, biases])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = False  #You can make this trainable if you want.
    updated_model.summary()
    return updated_model


In [22]:
vgg16_model = VGG16(include_top=False, weights='imagenet')
vgg16_updated = change_input_layer(vgg16_model, 1, 48, 48, 1)

['input_4', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool']
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        640       
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (

In [23]:
final_vgg16 = Sequential()
final_vgg16.add(vgg16_updated)
final_vgg16.add(Flatten())

In [105]:
current_path = None
all_features = []
files = []

for img, batch_paths in tqdm.tqdm(dataset):
    batch_features = final_vgg16(img)
    batch_features = tf.reshape(batch_features, 
                              (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if path != current_path and current_path is not None:
            #output_path = current_path.decode().replace('.avi', '.npy')
            #np.save(output_path, all_features)
            all_features = []
            
        current_path = path
        all_features.append(features)
        files.append(current_path)



14it [00:30,  2.28s/it]

face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected


38it [01:30,  2.47s/it]

face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected


73it [03:00,  2.66s/it]

...

84it [03:29,  2.24s/it]

face wasn't detected


85it [03:32,  2.40s/it]

face wasn't detected


161it [07:04,  2.64s/it]


tf.Tensor([b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1962\\S048-001.avi'], shape=(1,), dtype=string)


In [51]:
img, file_name = dataset

<class 'str'>


InvalidArgumentError: TypeError: Can't convert object to 'str' for 'filename'
Traceback (most recent call last):

  File "c:\Users\cdr03\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "c:\Users\cdr03\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\cdr03\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\cdr03\AppData\Local\Temp\ipykernel_34232\237410190.py", line 27, in frame_generator
    face_image_frame = detect_and_crop_face(frame)

  File "C:\Users\cdr03\AppData\Local\Temp\ipykernel_34232\3008466000.py", line 3, in detect_and_crop_face
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

TypeError: Can't convert object to 'str' for 'filename'


	 [[{{node PyFunc}}]] [Op:IteratorGetNex